In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ## Data Cleaning and Preprocessing

# In[44]:


# Import dependencies
import pandas as pd
import datetime
import time
#from sqlalchemy import create_engine
#from config import db_password


# In[45]:


# Read in files
df = pd.read_csv('Resources/bitcoin_data.zip')


# In[46]:


# Display initial data
df.head(10)


# In[47]:


# Drop all nulls
# df = df.dropna()


# In[48]:


# keep timestamp, high, low, weighted_price
df = df[['Timestamp', 'High', 'Low', 'Volume_(BTC)', 'Weighted_Price']].reset_index(drop=True)
df


# In[49]:


# preprocessing notes-- consider omitting data prior to Jan 1st, 2012 due to higher volatility 
# earlier in bitcoin's lifecycle

# converting Timestamp column to str datatype
df['str_timestamp'] = [str(timestamp) for timestamp in df['Timestamp']]
df['int_timestamp'] = df['str_timestamp'].astype('int32', copy=True)

# filtering df to only include rows after 01/01/2012
df_cleaned_filtered = df.loc[df['int_timestamp'] >= 1325391360]
df_cleaned_filtered.head(2)


# In[50]:


# context manager to suppress 1 time SettingWithCopyWarning; alternatively call .loc after timestamp conversion to avoid error
pd.reset_option('mode.chained_assignment')
with pd.option_context('mode.chained_assignment', None):
    df_cleaned_filtered['UTC_time'] = df_cleaned_filtered['int_timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x))

# .strftime('%Y-%m-%d %H:%M:%S')
    
# Convert from Unix to UTC time
#df_cleaned_filtered['UTC_time'] = df_cleaned_filtered['int_timestamp'].\
#    apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


# In[51]:


# dropping all null values, followed by evaluation of which time periods had the greatest data loss
final_df = df_cleaned_filtered.dropna().reset_index(drop=True)

# finding percentage of data loss (null values)
btotalLength = (len(df_cleaned_filtered))
bpartialLength = (len(final_df))
bValuesDropped = btotalLength - bpartialLength
bitstampDiff = (btotalLength - bpartialLength) / btotalLength * 100 
totalDays = round(btotalLength/1440,2)
daysDropped = round(bValuesDropped/1440,2)

print(f"Percentage of total Bitstamp data lost due to dropping rows with NaN values: {bitstampDiff:.2f}%")
print(f"Ratio of Bitstamp rows dropped: {bValuesDropped:,}/{btotalLength:,}")
print(f"Total dataset contains {totalDays} days worth of data. Due to data loss, we had to drop {daysDropped} days worth of data")


# In[52]:


# calculating UTC time deltas
final_df['time_delta'] = (final_df['UTC_time'] - final_df['UTC_time'].shift())

# creating mask to filter deltas
mask = final_df['time_delta'].apply(lambda x: True if x <= pd.Timedelta('0 days 00:01:00') else False)

# invert mask to create df with significant data loss
deltas_df = final_df[~mask]


# In[53]:


# shows longest periods of data loss
sorted_deltas_df = deltas_df.sort_values('time_delta', ascending=False, ignore_index=False)
sorted_deltas_df


# In[54]:


# shows periods of data loss where weighted price was highest
sorted_prices_df = deltas_df.sort_values('Weighted_Price', ascending=False, ignore_index=False)
sorted_prices_df


# In[55]:


# Machine learning section
# possible implementation ideas:
# 1. time series forecasting bitcoin price with ARIMA(Auto Regressive Integrated Moving Average)
# Rational for using time series: the 4 components of time series are trends, seasonality (consistent trends that span 1 calender year), noise/irregularity, and cyclicity(trends that can be observed in <1 or >1 year); this is perfect for tradeable assets 
# Qualifying criteria: the dataset must be stationary in order to more accurately fit a time series model. Stationary means that there should be a constant mean with constant std deviation or variance
# We can test if a time series is stationary by plotting the moving avg and/or movning std dev. to see if it changes over time. These results can be supported with an ADCF test
# if testing reveals our time series is NOT stationary, we can perform a variety of transformations to change it (log transform, sq rt, exponential decay etc..)
# SARIMA is ARIMA with a seasonality component. 


# ### PGAdmin SQL Schema

# In[56]:


# db schema thoughts: can create multiple tables for each currency (1 for btc, 1 ethereum, 1 xrp)
# can also add in additional tables down the line to support our analysis (table with google analytics, table with twitter search results for sentiment analysis)


# In[57]:


# View final dataset column names
final_df.head(5)


# In[58]:


# Rename columns to be sql friendly
final_df.rename({'Timestamp':'timestamp',
                  'High':'high',
                  'Low':'low',
                  'Volume_(BTC)':'volume',
                  'Weighted_Price':'weightedPrice',
                  'str_timestamp':'timestampSTR',
                  'int_timestamp':'timestampINT',
                  'UTC_time':'timeUTC',
                  'time_delta':'timeDelta'
                 }, axis='columns', inplace=True)


# In[59]:


# Converting timedelta to string
final_df = final_df.astype({'timeDelta':'str'})


# In[60]:


# Checking datatypes
final_df.dtypes


# In[61]:


# Verifying final dataframe
final_df.head(2)


